In [2]:
import pandas as pd

data = pd.read_csv("/scratch/rnd-rojas/assembly_stephanie/data/views.csv")
data

,Species,Running Position,Binned Coverage,view
0,Clostridium_perfringensATCC_13124,0,51.79,0
1,Clostridium_perfringensATCC_13124,10000,2362.99,0
2,Clostridium_perfringensATCC_13124,20000,51.65,0
3,Clostridium_perfringensATCC_13124,30000,50.41,0
4,Clostridium_perfringensATCC_13124,40000,51.51,0
...,...,...,...,...
10512,Thermobacillus_composti_KWC4,4306343,1004.75,1
10513,Thermobacillus_composti_KWC4,4316343,890.15,1
10514,Thermobacillus_composti_KWC4,4326343,798.36,1
10515,Thermobacillus_composti_KWC4,4336343,869.11,1


In [21]:
import altair as alt
import pandas as pd

# Prevent error about exceeding 5000 rows of data
alt.data_transformers.disable_max_rows()

data = pd.read_csv("/scratch/rnd-rojas/assembly_stephanie/data/views.csv")
df = pd.DataFrame(data, columns=['Species', 'Running Position', 'Binned Coverage', 'view'])

step = 30
overlap = 1

color_scheme='rainbow'


alt.Chart(df, height=step).transform_joinaggregate(
    mean_cov='mean(Binned Coverage)', groupby=['Species']
).transform_bin(
    ['bin_max', 'bin_min'], 'Binned Coverage'
).transform_aggregate(
    value='count()', groupby=['Species', 'mean_cov', 'bin_min', 'bin_max']
).transform_impute(
    impute='value', groupby=['Species', 'mean_cov'], key='bin_min', value=0
).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='lightgray',
    strokeWidth=0.5
).encode(
    alt.X('bin_min:Q', bin='binned', title='Binned Coverage'),
    alt.Y(
        'value:Q',
        scale=alt.Scale(range=[step, -step * overlap]),
        axis=None
    ),
    alt.Fill(
        'mean_cov:Q',
        legend=None,
        scale=alt.Scale(domain=[0, 4000], scheme='rainbow')
    )
).facet(
    row=alt.Row(
        'Species:N',
        title=None,
        header=alt.Header(labelAngle=0, labelAlign='right')
    )
).properties(
    title='Distribution of Binned Coverage',
    bounds='flush'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)

alt.FacetChart(...)